<a href="https://colab.research.google.com/github/shiernee/AI_Tutorial/blob/main/AI_Workshop_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Boston Housing Price Prediction with Feed Forward Neural Networks

Let's start with using a fully-connected neural network to do predict housing prices. 

For the Boston housing dataset, we get 506 rows of data, with 13 features in each. Our task is to build a regression model that takes these 13 features as input and output a single value prediction of the "median value of owner-occupied homes (in $1000)."

Now, we load the dataset. Loading the dataset returns four NumPy arrays:

* The `train_images` and `train_labels` arrays are the *training set*—the data the model uses to learn.
* The model is tested against the *test set*, the `test_images`, and `test_labels` arrays.

## 1. Import Data

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

#A bunch is you remember is a dictionary based dataset.  Dictionaries are addressed by keys. 
#Let's look at the keys. 
print(boston.keys())


In [ ]:
#DESCR sounds like it could be useful. Let's print the description.
print(boston['DESCR'])

# 2. Data Splitting


In [ ]:
from tensorflow import keras
(train_features, train_labels), (test_features, test_labels) = keras.datasets.boston_housing.load_data()


### **Exercise**

How many number of samples in train and test datasets respectively.

In [ ]:
# type your code here


In [ ]:
#@title Solution
print('train_features: ', len(train_features))
print('test_features: ', len(test_features))


### 3. Build the model

Building the neural network requires configuring the layers of the model, then compiling the model. First we stack a few layers together using `keras.Sequential`. Next we configure the loss function, optimizer, and metrics to monitor. These are added during the model's compile step:

* *Loss function* - measures how accurate the model is during training, we want to minimize this with the optimizer.
* *Optimizer* - how the model is updated based on the data it sees and its loss function.
* *Metrics* - used to monitor the training and testing steps.

Let's build a network with 1 hidden layer of 20 neurons, and use mean squared error (MSE) as the loss function (most common one for regression problems):

In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Dense

def build_model():
    model = keras.Sequential([
        Dense(20, activation=tf.nn.relu, input_shape=[len(train_features[0])]),
        Dense(1)
    ])

    model.compile(optimizer=tf.optimizers.Adam(), 
                  loss='mse',
                  metrics=['mse'])
    return model


model = build_model()

### 4. Train the model

Training the neural network model requires the following steps:

1. Feed the training data to the model—in this example, the `train_features` and `train_labels` arrays.
2. The model learns to associate features and labels.
3. We ask the model to make predictions about a test set—in this example, the `test_features` array. We verify that the predictions match the labels from the `test_labels` array. 

To start training,  call the `model.fit` method—the model is "fit" to the training data:

In [ ]:
import pandas as pd
import numpy as np

# this helps makes our output less verbose but still shows progress
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(train_features, train_labels, epochs=1000, verbose=0, validation_split = 0.1,
                    callbacks=[early_stop, PrintDot()])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

# show RMSE measure to compare to Kaggle leaderboard on https://www.kaggle.com/c/boston-housing/leaderboard
rmse_final = np.sqrt(float(hist['val_mse'].tail(1)))
print()
print('Final Root Mean Square Error on validation set: {}'.format(round(rmse_final, 3)))

Now, let's plot the loss function measure on the training and validation sets. The validation set is used to prevent overfitting ([learn more about it here](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit)). However, because our network is small, the training convergence without noticeably overfitting the data as the plot shows.

In [ ]:
import matplotlib.pyplot as plt

def plot_history():
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [Thousand Dollars$^2$]')
    plt.plot(hist['epoch'], hist['mse'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'], label = 'Val Error')
    plt.legend()
    plt.ylim([0,100])

plot_history()

## 5. Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
_, mse = model.evaluate(test_features, test_labels)
rmse = np.sqrt(mse)
print('Root Mean Square Error on test set: {}'.format(round(rmse, 3)))


# **Part 2. Design simple ANN model to predict SGA baby**
## Classification of Normal vs. Small baby

Task: Create a ANN model to predict the normal / small baby based on the second trimesters data. 

**Features:**
1. Age
2. Ethnics
3. Head Circumference
4. Abdominal Circumference 
2. Femur 

**Label:**
1. Labels


In [ ]:
# type your code here

# Solutions
Please try to do it yourself first. 

### Step1: Import data
 

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/shiernee/AI_Tutorial/main/fetal_dataset.csv'
data = pd.read_csv(url, delimiter=',')


In [ ]:
X = data[['Age', 'Ethnics', 'HC', 'AC', 'FL']]
y = data['labels (0-healthy, 1-SGA)']

### Normalize your data (Optional)

In [ ]:
import numpy as np
# get per-feature statistics (mean, standard deviation) from the training set to normalize by
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X_features = (X - X_mean) / X_std

## Step2: Data Splitting. 
Refer to this. <br> 
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html


In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(X_features, y, test_size=0.33, random_state=42)

## Step3: Build Model 


In [ ]:
input_dim = int(train_features.size/train_features.shape[0]) 

def build_model():
    model = keras.Sequential([
        Dense(20, activation=tf.nn.relu, input_shape=[input_dim]),
        # output a softmax to squash the matrix into output probabilities
        Dense(2, activation='softmax')
    ])

    model.compile(optimizer=tf.optimizers.Adam(), 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


model = build_model()

## Step4: Train Model


In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)

history = model.fit(train_features, train_labels, epochs=1000, verbose=0, validation_split = 0.1,
                    callbacks=[early_stop, PrintDot()])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch


# show accuracy measure 
train_acc = float(hist['accuracy'].tail(1))
val_acc = float(hist['val_accuracy'].tail(1))
print()
print('Train accuracy:', train_acc)
print('Validation accuracy:', val_acc)

In [ ]:
def plot_history():
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [Thousand Dollars$^2$]')
    plt.plot(hist['epoch'], hist['accuracy'], label='Train Accuracy')
    plt.plot(hist['epoch'], hist['val_accuracy'], label = 'Val Accuracy')
    plt.legend()
    plt.ylim([0,1])

plot_history()

As the model trains, the loss and accuracy metrics are displayed. This model reaches an training accuracy of about 71% and validation accuracy 62% on the training data.

## 5. Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
test_loss, test_acc = model.evaluate(test_features, test_labels)

print('Test accuracy:', test_acc)

Often times, the accuracy on the test dataset is a little less than the accuracy on the training dataset. This gap between training accuracy and test accuracy is an example of *overfitting*. 

## Acknowledgements

The contents of the Part 1: Regression tutorial is inspired and based on Lex Friedman's [tutorial_deep_learning_basic.ipynb](https://colab.research.google.com/github/lexfridman/mit-deep-learning/blob/master/tutorial_deep_learning_basics/deep_learning_basics.ipynb#scrollTo=IysPmcOBHBE9) 
